In [3]:
!pip install transformers datasets peft evaluate accelerate bitsandbytes pandas -q

In [8]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import os
import argparse
from typing import Dict, Any, List

import torch
from datasets import load_dataset, Dataset, ClassLabel, Features, Value
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np
import pandas as pd

# -------------------------
# Configuration
# -------------------------
class SimpleArgs:
    csv = "mt_bench_training.csv"
    output_dir = "./model_adapter"
    model_name = "bert-base-uncased"  # <-- switched to encoder-only
    batch_size = 8
    epochs = 3
    lr = 5e-5
    seed = 42
    max_input_length = 512
    lora_r = 16
    lora_alpha = 32
    lora_dropout = 0.05
    save_total_limit = 2
    eval_steps = 10
    logging_steps = 10
    seed_data_split = 42
    test_size = 0.1
    validation_size = 0.1

args = SimpleArgs()

# -------------------------
# Utilities
# -------------------------
def build_input_texts_from_columns(examples: Dict[str, List], tokenizer) -> List[str]:
    text_inputs = []
    sep_token = tokenizer.sep_token if tokenizer.sep_token is not None else " "

    for i in range(len(examples["turn"])):
        turn = int(examples["turn"][i])
        q1 = str(examples.get("turn_1_query", [""])[i]).strip()

        if turn == 1:
            text = f"Query: {q1}"
        elif turn == 2:
            ans = str(examples.get("turn_1_answer", [""])[i]).strip()
            q2 = str(examples.get("turn_2_query", [""])[i]).strip()
            text = f"Query: {q1}{sep_token}Answer: {ans}{sep_token}Follow-up Query: {q2}"
        else:
            text = f"Query: {q1}"
        text_inputs.append(text)
    return text_inputs


def preprocess_function(examples, tokenizer, args):
    text_inputs = build_input_texts_from_columns(examples, tokenizer)

    model_inputs = tokenizer(
        text_inputs,
        max_length=args.max_input_length,
        truncation=True,
        padding=False,
    )
    model_inputs["labels"] = examples["label"]
    return model_inputs


# -------------------------
# Compute Metrics
# -------------------------
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        logits = preds[0]
    else:
        logits = preds

    pred_ids = np.argmax(logits, axis=1)
    acc = accuracy_metric.compute(predictions=pred_ids, references=labels)
    return {"accuracy": acc["accuracy"]}

# -------------------------
# Main Logic
# -------------------------
def main():
    torch.manual_seed(args.seed)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"--- Using device: {device.upper()} ---")

    if not os.path.exists(args.csv):
        raise FileNotFoundError(f"CSV file not found: {args.csv}")

    df = pd.read_csv(args.csv)

    unique_winners = sorted(df["winner"].unique().tolist())
    label2id = {label: i for i, label in enumerate(unique_winners)}
    id2label = {i: label for i, label in enumerate(unique_winners)}
    num_labels = len(unique_winners)
    print(f"Found {num_labels} unique labels: {unique_winners}")

    df['label'] = df['winner'].map(label2id)

    features = Features({
        'question_id': Value('int64'),
        'turn': Value('int64'),
        'turn_1_query': Value('string'),
        'turn_1_answer': Value('string'),
        'turn_2_query': Value('string'),
        'winner': Value('string'),
        'label': ClassLabel(names=unique_winners)
    })

    raw_all = Dataset.from_pandas(df, features=features)

    train_val_split = raw_all.train_test_split(
        test_size=args.test_size,
        seed=args.seed_data_split,
        stratify_by_column="label"
    )
    test_ds = train_val_split["test"]
    train_val_ds = train_val_split["train"]

    train_split = train_val_ds.train_test_split(
        test_size=args.validation_size,
        seed=args.seed_data_split,
        stratify_by_column="label"
    )
    train_ds = train_split["train"]
    val_ds = train_split["test"]

    print(f"Dataset splits created: train={len(train_ds)}, validation={len(val_ds)}, test={len(test_ds)}")

    tokenizer = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        args.model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
    )

    # For BERT, we usually target query/key/value/projection layers for LoRA
    target_modules = ["query", "key", "value", "dense"]

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        lora_dropout=args.lora_dropout,
        target_modules=target_modules,
    )

    model = get_peft_model(model, peft_config)
    print("Wrapped model with LoRA. Trainable parameters:")
    model.print_trainable_parameters()

    tokenized_train = train_ds.map(lambda examples: preprocess_function(examples, tokenizer, args), batched=True)
    tokenized_val = val_ds.map(lambda examples: preprocess_function(examples, tokenizer, args), batched=True)
    tokenized_test = test_ds.map(lambda examples: preprocess_function(examples, tokenizer, args), batched=True)

    columns_to_remove = ['question_id', 'turn', 'turn_1_query', 'turn_1_answer', 'turn_2_query', 'winner', 'label']
    tokenized_train = tokenized_train.remove_columns([col for col in columns_to_remove if col in tokenized_train.column_names])
    tokenized_val = tokenized_val.remove_columns([col for col in columns_to_remove if col in tokenized_val.column_names])
    tokenized_test = tokenized_test.remove_columns([col for col in columns_to_remove if col in tokenized_test.column_names])

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir=args.output_dir,
        eval_strategy="steps",
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.epochs,
        learning_rate=args.lr,
        save_total_limit=args.save_total_limit,
        fp16=torch.cuda.is_available(),
        logging_steps=args.logging_steps,
        eval_steps=args.eval_steps,
        save_strategy="steps",
        save_steps=args.eval_steps,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        seed=args.seed,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("--- Starting Training ---")
    trainer.train()
    print("--- Training Finished ---")

    print("\n--- Evaluating on the held-out Test Set ---")
    test_results = trainer.evaluate(eval_dataset=tokenized_test)
    print("Test Set Metrics:")
    print(test_results)

    print("\nSaving final PEFT adapter to:", args.output_dir)
    model.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)
    print("Done.")

# Run the main function
main()


--- Using device: CUDA ---
Found 6 unique labels: ['alpaca-13b', 'claude-v1', 'gpt-3.5-turbo', 'gpt-4', 'llama-13b', 'vicuna-13b-v1.2']
Dataset splits created: train=3348, validation=373, test=414


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1528181905.py:199: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--- Starting Training ---


Step,Training Loss,Validation Loss,Accuracy
10,1.971800,1.827908,0.203753
20,1.822500,1.773134,0.209115
30,1.709400,1.733328,0.230563
40,1.749100,1.706487,0.281501
50,1.787100,1.700598,0.281501
60,1.675800,1.697499,0.281501
70,1.625200,1.693757,0.281501
80,1.678400,1.690828,0.281501
90,1.698400,1.687885,0.281501
100,1.713500,1.684683,0.281501
